In [1]:
import numpy as np
from vgn.networks import load_network
import torch
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from vgn.dataset_voxel_grasp_pc import DatasetVoxelGraspPCOcc
from pathlib import Path
import random

np.random.seed(0)
torch.manual_seed(0)

In [13]:
device = "cpu"
scene = "0a0b691837b745568fa6f98d546027ed"
root = Path('data/pile/data_pile_train_constructed_4M_HighRes_radomized_views_GPG_only')
raw_root = Path('data/pile/data_pile_train_random_raw_4M_radomized_views/')
model_path = 'data/best_runs/23-05-03-01-42-37_dataset=data_pile_train_constructed_4M_HighRes_radomized_views_no_table,augment=False,net=6d_neu_grasp_pn_deeper,batch_size=32,lr=5e-05,PN_no_tab_deeper_DIMS_WITH_occ/best_neural_grasp_neu_grasp_pn_deeper_val_acc=0.9120.pt'
net = "neu_grasp_pn_deeper"
net_with_grasp_occ = True

net = load_network(model_path, device=device, model_type=net)

# NOTE: Renamed "grasps_with_clouds_gt".csv to "grasps_with_clouds.csv"
data = DatasetVoxelGraspPCOcc(root, raw_root, use_grasp_occ=net_with_grasp_occ)

Loading [neu_grasp_pn_deeper] model from data/best_runs/23-05-03-01-42-37_dataset=data_pile_train_constructed_4M_HighRes_radomized_views_no_table,augment=False,net=6d_neu_grasp_pn_deeper,batch_size=32,lr=5e-05,PN_no_tab_deeper_DIMS_WITH_occ/best_neural_grasp_neu_grasp_pn_deeper_val_acc=0.9120.pt


## Helper Functions

In [14]:
def visualize_plotly(geometries, camera_eye=[1.25, 1.25, 1.25]):
    '''
    geometries (numpy arrays): [Voxels, PCD1, PCD2, ... PCDN]
    If mesh:
    geometries [dictionary ; numpy arrays] : [Dict, Surface points] 

    Note: If you do not want to plot Voxels, pass `None`
    '''

    fig = make_subplots(
                        rows=1, cols=2,
                        specs=[[{'type': 'scene'}, {'type': 'scene'}]])
    # IF: TSDF VOXELS
    Voxels = VoxelData(geometries[0])
    voxels = go.Mesh3d(
            x=Voxels.vertices[0],
            y=Voxels.vertices[1],
            z=Voxels.vertices[2],
            i=Voxels.triangles[0],
            j=Voxels.triangles[1],
            k=Voxels.triangles[2]
            )

    fig.add_trace(voxels, row=1, col=1)
    # Surface points
    for i in range(1, len(geometries)):
        pcd = go.Scatter3d(
                        x=geometries[i][:,0], 
                        y=geometries[i][:,1], 
                        z=geometries[i][:,2], 
                        marker=go.scatter3d.Marker(size=4), 
                        opacity=0.8, 
                        mode='markers')
        # data.append(pcd)
        fig.add_trace(pcd, row=1, col=2)

    print("Generating figure")
    # fig = go.Figure(data=data)
    fig.update_xaxes(tickmode='linear')
    fig.show()
    return fig

In [17]:
def load_data(scene):
    index = random.choice(data.df[data.df.scene_id==scene].index)
    pc, y, grasp_query, occ_points, occ =  data[index]
    return pc, y, grasp_query, occ_points

# Load

## Load network

In [18]:
x, y, grasp_query, pos_occ = load_data(scene)
out = net(x, grasp_query, p_tsdf=pos_occ)
sdf = out[-1]

FileNotFoundError: [Errno 2] No such file or directory: 'data/pile/data_pile_train_random_raw_4M_radomized_views/occ_grasp_point_clouds_noisy/1076741.npz'

## Occupancy

In [ ]:
x, y, grasp_query, pos_occ = prepare_batch(batch, device) # <- Changed to predict only grasp quality (check inside)
        y_pred = select(net(x, grasp_query, p_tsdf=pos_occ))

In [ ]:
# # # Check if network gives correct occupancies
# x, y, z = torch.meshgrid(torch.linspace(start=-0.5, end=0.5 - 1.0 / 40, steps= 40), torch.linspace(start=-0.5, end=0.5 - 1.0 / 40, steps=40), torch.linspace(start=-0.5, end=0.5 - 1.0 / 40, steps=40))
#         # 1, self.resolution, self.resolution, self.resolution, 3
# pos = torch.stack((x, y, z), dim=-1).float().unsqueeze(0)
# pos = pos.view(1, 40 * 40 * 40, 3)
# occupancies_64K = net.renderer.model(pos, torch.Tensor(tsdf), only_occupancy=True)
# import mcubes
# import plotly.graph_objects as go

# vertices, triangles = mcubes.marching_cubes(occupancies_64K.view(40, 40, 40).detach().numpy(), 0.5)
# x, y, z = vertices.T
# i, j, k = triangles.T

In [ ]:
# fig = go.Figure(go.Mesh3d(x=x, y=y, z=z,
#                           i=i, j=j, k=k,
#                           intensity=z))
# fig.show()

## Rendering

In [ ]:
## Visualize TSDF with only GT pointcloud, occupied points 
visualize_plotly([tsdf.squeeze(), pc, points_occ])

## Visualize TSDF with GT pointcloud, occupied points, image plane and camera-pos
# points_occ_shifted = points_occ.clone().detach()
# points_occ_shifted[:, 0] = points_occ_shifted[:, 0] - 0.03 # Shifting x-axis
# points_occ_shifted[:, 1] = points_occ_shifted[:, 1] - 0.025 # Shifting y-axis
# visualize_plotly([tsdf, pc, points_occ_shifted, pixels_world[0], cam_position.reshape(1, 3)])

## Surface Rendering

In [ ]:
val = val.view(batch_size, -1, n_steps)
mask_0_not_occupied = val[:, :, 0] < 0
sign_matrix = torch.cat([torch.sign(val[:, :, :-1] * val[:, :, 1:]),
                                 torch.ones(batch_size, n_pts, 1)],
                                dim=-1)
cost_matrix = sign_matrix * torch.arange(n_steps, 0, -1).float()
values, indices = torch.min(cost_matrix, -1)
mask_sign_change = values < 0
mask_neg_to_pos = val[torch.arange(batch_size).unsqueeze(-1),torch.arange(n_pts).unsqueeze(-0), indices] < 0
mask = mask_sign_change & mask_neg_to_pos & mask_0_not_occupied
print("Mask truth:", sum(sum(mask==True)))

In [ ]:
def secant(tsdf, f_low, f_high, d_low, d_high, n_secant_steps,
                          ray0_masked, ray_direction_masked, tau, it=0):
    ''' Runs the secant method for interval [d_low, d_high].

    Args:
        d_low (tensor): start values for the interval
        d_high (tensor): end values for the interval
        n_secant_steps (int): number of steps
        ray0_masked (tensor): masked ray start points
        ray_direction_masked (tensor): masked ray direction vectors
        model (nn.Module): model model to evaluate point occupancies
        c (tensor): latent conditioned code c
        tau (float): threshold value in logits
    '''
    d_pred = - f_low * (d_high - d_low) / (f_high - f_low) + d_low
    for i in range(n_secant_steps):
        p_mid = ray0_masked + d_pred.unsqueeze(-1) * ray_direction_masked
        p_scaled_mid = p_mid/0.3 - 0.5

        with torch.no_grad():
            f_mid = (net.renderer.model(p_scaled_mid.view(1, -1, 3),  tsdf, 
                            only_occupancy=True)- tau).squeeze()
        ind_low = f_mid < 0
        ind_low = ind_low
        if ind_low.sum() > 0:
            d_low[ind_low] = d_pred[ind_low]
            f_low[ind_low] = f_mid[ind_low]
        if (ind_low == 0).sum() > 0:
            d_high[ind_low == 0] = d_pred[ind_low == 0]
            f_high[ind_low == 0] = f_mid[ind_low == 0]

        d_pred = - f_low * (d_high - d_low) / (f_high - f_low) + d_low
    return d_pred

In [ ]:
n = batch_size * n_pts
d_low = d_proposal.view(n, n_steps, 1)[torch.arange(n), indices.view(n)].view(batch_size, n_pts)[mask]
f_low = val.view(n, n_steps, 1)[torch.arange(n), indices.view(n)].view(
            batch_size, n_pts)[mask]
indices = torch.clamp(indices + 1, max=n_steps-1)
d_high = d_proposal.view(n, n_steps, 1)[torch.arange(n), indices.view(n)].view(batch_size, n_pts)[mask]
f_high = val.view(n, n_steps, 1)[torch.arange(n), indices.view(n)].view(batch_size, n_pts)[mask]

In [ ]:
ray0_masked = camera_world[mask]
ray_direction_masked = ray_vector[mask]

# Apply surface depth refinement step (e.g. Secant method)
d_pred = secant(torch.Tensor(tsdf),
    f_low, f_high, d_low, d_high, n_secant_steps, ray0_masked,
    ray_direction_masked, tau = 0.5)

# for sanity
d_pred_out = torch.ones(batch_size, n_pts)
d_pred_out[mask] = d_pred
d_pred_out[mask == 0] = np.inf
d_pred_out[mask_0_not_occupied == 0] = 0
d_i = d_pred_out

In [ ]:
def get_mask(tensor):
    ''' Returns mask of non-illegal values for tensor.

    Args:
        tensor (tensor): Numpy or Pytorch tensor
    '''
    # tensor, is_numpy = to_pytorch(tensor, True)
    mask = ((abs(tensor) != np.inf) & (torch.isnan(tensor) == False))
    mask = mask.bool()
    # if is_numpy:
    #     mask = mask.numpy()

    return mask

In [ ]:
# p_proposal[mask, :, :].shape

In [ ]:
mask_zero_occupied = d_i == 0
d_i = d_i.detach()

# Get mask for predicted depth
mask_pred = get_mask(d_i).detach()

with torch.no_grad():
    dists = torch.ones_like(d_i)
    dists[mask_pred] = d_i[mask_pred]
    dists[mask_zero_occupied] = 0.
    network_object_mask = mask_pred & ~mask_zero_occupied
    network_object_mask = network_object_mask[0]
    dists = dists[0]

# Project depth to 3d poinsts
camera_world = camera_world.reshape(-1, 3)
ray_vector = ray_vector.reshape(-1, 3)

points = camera_world + ray_vector * dists.unsqueeze(-1)
points = points.view(-1, 3)

In [ ]:
surface_mask = network_object_mask.view(-1)
surface_points = points[surface_mask]
N = surface_points.shape[0]
surface_points_neig = surface_points + (torch.rand_like(surface_points) - 0.5) * 0.01
pp = torch.cat([surface_points, surface_points_neig], dim=0)
pp_scaled = (pp - torch.min(pp))/(torch.max(pp) - torch.min(pp)) - 0.5
g = net.renderer.model.gradient(torch.Tensor(tsdf), pp_scaled.view(1, -1, 3)).permute(1, 0, 2) #Unisurf in [n_pts, 1, 3]
# print(g.size())
normals_ = g[:, 0, :] / (g[:, 0, :].norm(2, dim=1).unsqueeze(-1) + 10**(-5)) 
diff_norm =  torch.norm(normals_[:N] - normals_[N:], dim=-1)
# surface_points = p_proposal[mask, :, :].view(-1, 3)

## Render Visual

In [ ]:
visualize_plotly([tsdf.squeeze(), pc, surface_points.reshape(-1, 3)])